# NLP_GoingDeeper | 03. 워드임베딩
- 기계는 텍스트보다 수치화된 숫자를 더 잘 처리할 수 있기때문에, 기계가 자연어 처리를 원활히 할 수 있도록, 전처리 과정에서 텍스트를 벡터로 변환하는 벡터화(Vectorization) 과정을 거치게 됩니다. 
- 자연어 처리에서 단어를 벡터화하는 방법 중 하나인 워드 임베딩을 알아봅니다.

---

# 0. 벡터화기법 
- Bag of words / DTM(Document-Term Matrix)
    - BoW는 단어의 순서를 고려하지 않고, 단어의 등장 빈도(frequency)만을 고려해서 단어를 벡터화하는 방법입니다. 
    - BoW를 사용하여 문서 간 유사도를 비교하기 위한 행렬을 DTM(문서 단어 행렬, Document-Term Matrix)라고 합니다. 이는 문서를 행으로, 단어를 열으로 구성한 행렬입니다. 
    - BoW 방법을 사용할 때는 주로 불필요한 단어를 제거하거나, 표현은 다르지만 같은 단어를 통합시켜주는 정규화와 같은 전처리를 합니다. BoW는 단어를 카운트 하는 방법론이기 때문에 불필요한 카운트는 미리 배제하는 것이 좋기 때문입니다. 
    - 이때, DTM의 문서 벡터나 단어 벡터는 대부분의 값이 0이라는 특징을 가지는데, 이러한 벡터를 **희소 벡터(sparse vector)** 라고 합니다. 
    - 문서의 수나 단어의 수가 많아질수록 행렬에서 대부분의 값이 0이 되는 희소 문제는 점점 심화된다는 특징이 있습니다. 
    - DTM에서 열의 개수는 전체 텍스트에서 한 번이라도 등장한 단어의 개수입니다. 이처럼 중복 카운트를 배제한 단어들의 집합(set)을 자연어 처리에서는 **단어장(vocabulary)** 라고 합니다. 
    - 참고 : [BoW 설명 영상](https://youtu.be/dKYFfUtij_U)
- TF-IDF(Term Frequency - Inverse Document Frequency)
    - DTM을 이용해서 문서의 유사도를 비교하는 경우, 두 문서에서 공통적으로 등장하는 단어가 많으면 그 두 문서는 유사하다고 판단하면 됩니다. 
    - 하지만 별로 중요치 않은(유사도 비교에 유의미하지 않은) 단어임에도, 모든 문서에서 공통적으로 등장하는 단어가 있습니다. 예를 들어 관사 'the'의 경우, 거의 모든 문서에 많이 등장하지만 'the' 가 등장하는 모든 문서가 유사도가 높다고 할 수는 없습니다. 
    - 그래서 단어마다 중요 가중치를 다르게 주는 방법인 TF-IDF가 등장했습니다.
    - To find how relevant a term is in a document   
    - 참고 : [TF-IDF 설명](https://youtu.be/meEchvkdB1U)
    
- 원-핫 인코딩(one-hot encoding)
    - 모든 단어의 관계를 독립적으로 정의하는 원-핫 인코딩 방식도 있습니다. 
    - 원-핫 인코딩을 하기 위해서는 우선 갖고 있는 텍스트 데이터에서 단어들의 집합인 단어장(vocabulary)를 만듭니다. 그 후 단어장에 있는 모든 단어에 대해서 1부터 V(단어장의 크기)까지 고유한 정수를 부여합니다(또는 정수를 0번부터 부여하기도 합니다). 이 정수는 단어장에 있는 각 단어의 일종의 인덱스 역할을 합니다. 각 단어는 V차원의 벡터로 표현되는데, 해당 단어의 인덱스 위치만 1이고 나머지는 전부 0의 값을 갖는 벡터가 됩니다. 
    - 숫자 부여에 정해진 규칙은 없지만, 관례적으로는 빈도수가 높은 단어들부터 낮은 숫자를 부여합니다. 이렇게 하면, 아주 큰 정수가 부여된 단어는 빈도수가 아주 낮은 단어라는 뜻이고, 그 단어는 중요하지 않은 단어일 확률이 높습니다. 그러므로 추가적인 전처리로 정수가 아주 큰 숫자는 단어장에서 제거할 수도 있습니다. 
    - 이처럼 원-핫 인코딩을 통해 얻은 벡터를 원-핫 벡터(one-hot vector)라고 합니다. 
    - TF(Term Frequency)란 문장을 구성하는 단어들의 원-핫 벡터들을 모두 더해서 문장의 단어 갯수로 나눈 것과 같습니다.
    


In [1]:
mkdir -p ~/aiffel/GoingDeeper/DATA/word_embedding

## 벡터화 실습 : 원-핫 인코딩 구현
- 데이터 전처리
    - 한글과 공백을 제외한 특수문자 제거
        - 자음의 범위는 'ㄱ-ㅎ', 모음의 범위는 'ㅏ-ㅣ' 로 지정할 수 있습니다. 
        - 완성형 한글의 범위는 '가-힣'으로 지정할 수 있습니다.
        - 한글과 공백을 제외하고 모든 문자를 표현하는 정규 표현식(regex) : [^ㄱ-하-ㅣ가-힣 ]
        - 참고 : [Hangul Compatibility Jamo](https://www.unicode.org/charts/PDF/U3130.pdf), [Hangul Syllables](https://www.unicode.org/charts/PDF/UAC00.pdf)

- 토큰화
    - 단어장을 구성하기 위해서는 단어장의 원소인 토큰(token)이라는 단어를 정해주어야 합니다. 
    - 한국어는 주로 형태소 분석기를 통해 토큰의 단위를 나눠줍니다. 
    - 여기서는 KoNLPy에 내장된 Okt 형태소 분석기를 사용해보겠습니다. 
    - 참고 : [KoNLPy Okt](https://konlpy.org/en/latest/api/konlpy.tag/#okt-class)

- 단어장 생성
    - 빈도수가 높은 단어일수록 낮은 정수를 부여하겠습니다. 
    - 파이썬의 Counter 서브클래스를 사용하여 단어의 빈도를 카운트합니다.
    - vocab에는 단어가 key로, 단어에 대한 빈도수가 value로 저장되어있습니다. 
    - most_common()은 상위 빈도수를 가진 단어를 주어진 수만큼 리턴합니다. 등장 빈도 수 상위 5개의 단어만 단어장으로 저장해보겠습니다. 
    - 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여하면, 최종 단어장 완성입니다.
    - 참고 : [파이썬 collections.Counter](https://docs.python.org/3/library/collections.html#collections.Counter)

- 원-핫 벡터 만들기
    - 원-핫 인코딩을 하는 함수를 만들어 각 단어를 원-핫 벡터로 만듭니다. 
    - 케라스를 통한 원-핫 인코딩
        - 원-핫 인코딩을 지원하는 여러 패키지 중 텐서플로의 케라스 API를 사용해봅니다.
        - 단어장을 만드는 역할의 Tokenizer와 원-핫 인코딩을 위한 to_categorical을 사용합니다. 
        - 케라스 토크나이저를 사용하면 주어진 텍스트로부터 단어장을 만들고, 단어장의 각 단어에 고유한 정수를 맵핑해줍니다.
        - 단어장의 크기를 vocab_size 변수에 저장합니다. 케라스 토크나이저는 각 단어에 고유한 정수를부여할 때, 숫자 1부터 부여하지만 실제로 자연어 처리를 할 때는 특별 토큰으로 0번 단어로 단어장에 추가로 사용하는 경우가 많습니다. 주로 0번은 패딩(padding)작업을 위한 패팅 토큰으로 사용됩니다. 따라서 vocab_size에 1을 더해줍니다. 
        - 케라스 토크나이저에 단어장이 저장되었으므로, 단어장에 속한 단어들로 구성된 텍스트 시퀀스는 케라스 토크나이저를 통해 정수 시퀀스로 변환될 수 있습니다. 
        - 정수시퀀스는 to_categorical()을 사용해 원-핫 벡터의 시퀀스로 변환할 수 있습니다. 원-핫 인코딩 결과 각 단어가 단어장의 크기인 56 차원의 벡터로 변환되었습니다. 

In [2]:
#- 한국어 형태소 분석기 패키지 설치
! pip install konlpy

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import re 
from konlpy.tag import Okt
from collections import Counter

In [4]:
text_0 = "시드가 날 로켓이 묶었을때 우디가 포기하던? 너희들이 우디를 트럭에서 던졌을때 우디가 포기하던? 그대 우디는 포기하지 않았어!!"
text_1 = "두목, 돌과 비와 꽃이 하는 말을 들을 수 있다면 얼마나 좋겠어요. 두목, 언제면 우리 귀가 뚫릴까요! 언제면 우리가 팔을 벌리고 만물을 안을 수 있을까요? 두목, 어떻게 생각해요? 당신이 읽는 책에는 뭐라고 쓰여 있습니까?"
text_2 = "웃어라 온 세상이 너와 함께 웃을 것이다. 울어라 너 혼자 울게 될 것이다"

In [5]:
#- 특수문자 제거
reg = re.compile("[^ㄱ-하-ㅣ가-힣 ]")
text_0 = reg.sub('', text_0)
text_1 = reg.sub('', text_1)
text_2 = reg.sub('', text_2)
print(f'text_0 : {text_0}\ntext_1 : {text_1}\ntext_2 : {text_2}')

text_0 : 시드가 날 로켓이 묶었을때 우디가 포기하던 너희들이 우디를 트럭에서 던졌을때 우디가 포기하던 그대 우디는 포기하지 않았어
text_1 : 두목 돌과 비와 꽃이 하는 말을 들을 수 있다면 얼마나 좋겠어요 두목 언제면 우리 귀가 뚫릴까요 언제면 우리가 팔을 벌리고 만물을 안을 수 있을까요 두목 어떻게 생각해요 당신이 읽는 책에는 뭐라고 쓰여 있습니까
text_2 : 웃어라 온 세상이 너와 함께 웃을 것이다 울어라 너 혼자 울게 될 것이다


In [6]:
#- 토큰화
okt = Okt()
tokens_0 = okt.morphs(text_0) #- Toy Story
tokens_1 = okt.morphs(text_1) #- 그리스인 조르바
tokens_2 = okt.morphs(text_2) #- 올드보이
print(f'tokens_0 : {tokens_0}\ntokens_1 : {tokens_1}\ntokens_2 : {tokens_2}')

tokens_0 : ['시드', '가', '날', '로켓', '이', '묶었을', '때', '우디', '가', '포기', '하던', '너희', '들', '이', '우디', '를', '트럭', '에서', '던졌을', '때', '우디', '가', '포기', '하던', '그대', '우디', '는', '포기', '하지', '않았어']
tokens_1 : ['두목', '돌', '과', '비', '와', '꽃', '이', '하는', '말', '을', '들을', '수', '있다면', '얼마나', '좋겠어요', '두목', '언제', '면', '우리', '귀가', '뚫릴까', '요', '언제', '면', '우리', '가', '팔', '을', '벌리고', '만물', '을', '안', '을', '수', '있을까요', '두목', '어떻게', '생각', '해요', '당신', '이', '읽는', '책', '에는', '뭐라고', '쓰여', '있습니까']
tokens_2 : ['웃어라', '온', '세상', '이', '너', '와', '함께', '웃', '을', '것', '이다', '울어라', '너', '혼자', '울', '게', '될', '것', '이다']


In [7]:
#- 단어장 생성
vocab_0 = Counter(tokens_0)
vocab_1 = Counter(tokens_1)
vocab_2 = Counter(tokens_2)
print(f'vocab_0 : {vocab_0}\nvocab_1 : {vocab_1}\nvocab_2 : {vocab_2}')

vocab_0 : Counter({'우디': 4, '가': 3, '포기': 3, '이': 2, '때': 2, '하던': 2, '시드': 1, '날': 1, '로켓': 1, '묶었을': 1, '너희': 1, '들': 1, '를': 1, '트럭': 1, '에서': 1, '던졌을': 1, '그대': 1, '는': 1, '하지': 1, '않았어': 1})
vocab_1 : Counter({'을': 4, '두목': 3, '이': 2, '수': 2, '언제': 2, '면': 2, '우리': 2, '돌': 1, '과': 1, '비': 1, '와': 1, '꽃': 1, '하는': 1, '말': 1, '들을': 1, '있다면': 1, '얼마나': 1, '좋겠어요': 1, '귀가': 1, '뚫릴까': 1, '요': 1, '가': 1, '팔': 1, '벌리고': 1, '만물': 1, '안': 1, '있을까요': 1, '어떻게': 1, '생각': 1, '해요': 1, '당신': 1, '읽는': 1, '책': 1, '에는': 1, '뭐라고': 1, '쓰여': 1, '있습니까': 1})
vocab_2 : Counter({'너': 2, '것': 2, '이다': 2, '웃어라': 1, '온': 1, '세상': 1, '이': 1, '와': 1, '함께': 1, '웃': 1, '을': 1, '울어라': 1, '혼자': 1, '울': 1, '게': 1, '될': 1})


In [8]:
#- 빈도수 상위 5개 단어만 단어장으로 저장
vocab_size = 5
vocab_0 = vocab_0.most_common(vocab_size)
vocab_1 = vocab_1.most_common(vocab_size)
vocab_2 = vocab_2.most_common(vocab_size)
print(f'vocab_0 : {vocab_0}\nvocab_1 : {vocab_1}\nvocab_2 : {vocab_2}')

vocab_0 : [('우디', 4), ('가', 3), ('포기', 3), ('이', 2), ('때', 2)]
vocab_1 : [('을', 4), ('두목', 3), ('이', 2), ('수', 2), ('언제', 2)]
vocab_2 : [('너', 2), ('것', 2), ('이다', 2), ('웃어라', 1), ('온', 1)]


In [9]:
#- 최종 단어장 생성; 정수 인덱스 부여(1부터 시작) 
word2idx_0 = {word[0]:index+1 for index, word in enumerate(vocab_0)}
word2idx_1 = {word[0]:index+1 for index, word in enumerate(vocab_1)}
word2idx_2 = {word[0]:index+1 for index, word in enumerate(vocab_2)}
print(f'word2idx_0 : {word2idx_0}\nword2idx_1 : {word2idx_1}\nword2idxb_2 : {word2idx_2}')

word2idx_0 : {'우디': 1, '가': 2, '포기': 3, '이': 4, '때': 5}
word2idx_1 : {'을': 1, '두목': 2, '이': 3, '수': 4, '언제': 5}
word2idxb_2 : {'너': 1, '것': 2, '이다': 3, '웃어라': 4, '온': 5}


In [10]:
#- 원-핫 벡터 만들기
def one_hot_encoding(word, word2index):
    one_hot_vector = [0]*(len(word2index))
    index = word2index[word]
    one_hot_vector[index-1] = 1
    return one_hot_vector

one_hot_encoding("두목", word2idx_1)

[0, 1, 0, 0, 0]

In [11]:
#- 케라스를 통한 tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

text = [text_0, text_1, text_2]
t = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1 #- 단어장 크기 저장 

print(t.word_index) #- 각 단어에 대한 인코딩 결과 출력

{'두목': 1, '우디가': 2, '포기하던': 3, '수': 4, '언제면': 5, '것이다': 6, '시드가': 7, '날': 8, '로켓이': 9, '묶었을때': 10, '너희들이': 11, '우디를': 12, '트럭에서': 13, '던졌을때': 14, '그대': 15, '우디는': 16, '포기하지': 17, '않았어': 18, '돌과': 19, '비와': 20, '꽃이': 21, '하는': 22, '말을': 23, '들을': 24, '있다면': 25, '얼마나': 26, '좋겠어요': 27, '우리': 28, '귀가': 29, '뚫릴까요': 30, '우리가': 31, '팔을': 32, '벌리고': 33, '만물을': 34, '안을': 35, '있을까요': 36, '어떻게': 37, '생각해요': 38, '당신이': 39, '읽는': 40, '책에는': 41, '뭐라고': 42, '쓰여': 43, '있습니까': 44, '웃어라': 45, '온': 46, '세상이': 47, '너와': 48, '함께': 49, '웃을': 50, '울어라': 51, '너': 52, '혼자': 53, '울게': 54, '될': 55}


In [12]:
#- 케라스를 통한 texts to sequences

encoded = t.texts_to_sequences(text)
print(encoded)

[[7, 8, 9, 10, 2, 3, 11, 12, 13, 14, 2, 3, 15, 16, 17, 18], [1, 19, 20, 21, 22, 23, 24, 4, 25, 26, 27, 1, 5, 28, 29, 30, 5, 31, 32, 33, 34, 35, 4, 36, 1, 37, 38, 39, 40, 41, 42, 43, 44], [45, 46, 47, 48, 49, 50, 6, 51, 52, 53, 54, 55, 6]]


In [13]:
#- 케라스를 통한 one-hot encoding
one_hot = to_categorical(encoded[0], num_classes = vocab_size)
print(one_hot)

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

# 1. 워드 임베딩
- 희소 벡터의 문제점 
    - 벡터 또는 행렬의 값이 대부분이 0으로 표현되는 방법을 희소 표현(sparse representation)이라고 합니다. 
    - DTM, TF-IDF, 원-핫 벡터는 단어장의 크기에 영향을 받는 희소 벡터라는 특징을 가집니다.
    - 원-핫 인코딩의 경우 단어장의 크기가 30,000이라면 30,000개의 모든 단어 벡터는 각각 30,000 차원의 벡터가 됩니다... 이 벡터들은 하나의 원소만 1이고 29,999개의 원소가 0의 값을 가집니다. 
    - 희소 벡터에는 차원의 저주(curse of dimensionality)의 문제가 있습니다. 
        - 같은 정보를 저차원과 고차원에 각각 표현한다고 가정할 때, 저차원에서는 정보의 밀도가 상대적으로 커지지만, 고차원에서는 정보가 흩어지며 밀도가 작아집니다. 
        - 정보의 밀도가 작아지는 것, 즉 차원이 커지는 것과 머신러닝 모델의 성능에는 연관관계가 있습니다. 
        - 참고 : [빅데이터: 큰 용량의 역습 - 차원의 저주(Curse of dimensionality](http://thesciencelife.com/archives/1001)
    - 원-핫 벡터의 경우, 단어 간의 의미적 유사성을 반영하지 못합니다. 
        - 벡터 간 유사도를 구하는 방법으로는 대표적으로 내적(inner product)가 있습니다.
        - 임의의 두개의 원-핫 벡터 간 내적(innder product)을 구하면 서로 직교(orthogonal)하여 그 값은 0 입니다. 이는 모든 원-핫 벡터의 상호 유사도가 0임을 의미하며, 결국 원-핫 벡터를 통해서는 단어 벡터 간 유사도를 구할 수 없음을 의미합니다. 
    - 이에 대한 대안으로 **'기계가 단어장 크기보다 적은 차원의 밀집 벡터(dense vector)를 학습'** 하는 **워드 임베딩(word embedding)** 이 제안되었습니다. 
    - 이를 통해 얻는 밀집 벡터는 각 차원이 0과 1이 아닌 다양한 실숫값을 가지며, 이 밀집 벡터를 임베딩 벡터(embedding vector)라고 합니다. 

---

- 원-핫 벡터의 값은 각 단어에 정수를 맵핑해주고, 이로부터 인코딩한다는 점에 표현 방식이 '수동'이라고 볼 수 있습니다. 
- 반면, 임베딩 벡터의 값은, 훈련 데이터로부터 어떤 모델을 학습하는 과정에서 '자동'으로 얻어지는데, 주로 언어 모델(Language Model)을 학습하는 가운데 얻어집니다.
- 워드 임베딩은 2003년 요슈아 벤지오(Yoshua Bengio)교수가 LPLM(Neural Probabilistic Language Model)이란 모델을 통해 제안했습니다. 
- 하지만 당시 이 모델은 학습 속도가 지나치게 느리다는 단점이 있었고, 2013년 구글은 NPLM을 개선하여 정밀도와 속도를 향상시킨 Word2Vec을 제안했습니다. 
- Word2Vec 이후로 FastText나 GloVe 등과 같은 임베딩 방법이 추가로 제안되었습니다. 
- 오늘은 **Word2Vec(Google), FastText(Facebook), GloVe(Stanford)** 세가지 워드 임베딩 방법을 톺아보겠습니다.
- 참고 : [워드 임베딩](https://wikidocs.net/33520)
    - Word2Vec
        - provided by Google, released in 2013 
        - trained on Google News data
        - has 300 dimensions and is trained on 3million words from google news data
        - used skip-gram and negative sampling to build this model
        
    - GloVe
        - Global Vectors for words representation
        - provided by stanford
        - provided various models from 25, 50, 100, 200 to 300 dimensions based on 2, 6, 42, 840 billion tokens
        - used word-to-word co-occurrence to build this model
        - if two words co-occur many times, it means they have some linguistic or semantic similiarity

    - fastText
    
        - developed by Facebook
        - provided 3 models with 300 dimensions each

# 2. Word2Vec
## 2-1. 분포 가설
- [korean Word2Vec](https://word2vec.kr/search/) 는 한국어 데이터로 Word2Vec을 학습하여, 학습된 Word2Vec 벡터들로 연산한 결과를 제공하는 사이트입니다. 
    - 김+햄+단무지+쌀+게맛살+당근 => 우엉/Noun
    - 한국-서울+도쿄 => 일본/Noun
    - 단어들로 의미적 연산을 하는 것처럼 보입니다. 
    - 참고 : [Word2Vec 설명 영상](https://youtu.be/sY4YyacSsLc)

- 분포 가설(Distributional Hypothesis)
    - Word2Vec은 단어를 벡터로 표현하는 방법의 일종으로 저차원으로 이뤄져 있고, 단어의 의미를 여러 차원에 분산하여 표현한 벡터입니다. 
    - Word2Vec의 핵심 아이디어는 분포 가설(distributional hypothesis)를 따릅니다. 이 가설은 언어학자 John Rupert Firth의 인용으로 설명됩니다.
        - You shall know a word by the company it keeps (곁에 오는 단어들을 보면 그 단어를 알 수 있다.)
        - 이 인용은 어떤 단어들의 의미를 보려면 주변 단어들을 보라는 의미를 내포합니다. 
    - 분포 가설 : '비숫한 문맥에서 같이 등장하는 경향이 있는 단어들은 비슷한 의미를 가진다.'
    - 분포 가설에 따르는 Word2Vec은 같이 등장하는 경향이 적은 단어들에 비해 같이 등장하는 경향이 높은 단어들을 상대적으로 유사도가 높은 벡터로 만듭니다. 

## 2-2. CBoW (Continuous Bag of words)

- Word2Vec에는 크게 CBow와 'Skip-gram'이라는 두 가지 방법이 있습니다. 
    - CBoW는 주변에 있는 단어들을 통해 중간에 있는 단어들을 예측하는 방법입니다.
    - 반대로, Skip-Gram은 중간에 있는 단어로 주변 단어들을 예측하는 방법입니다. 
    - 메커니즘 자체는 거의 동일하기 때문에 CBow를 이해한다면 Skip-Gram도 쉽게 이해가 가능합니다. 
    - Because there are multiple contextual words, we average their corresponding word vectors, constructed by the multiplication of the input vector and the matrix W. 
    - Because the averaging stage smoothes over a lot of the distributional information, some people believe the CBOW model is better for small dataset.
---

- "I like natural language processing"
- 갖고 있는 코퍼스에 위와 같은 문장이 있다고 가정합니다. 
- CBoW는 중간에 있는 단어를 예측하는 방법이므로 {"i", "like", "language", "processing"} 으로부터 "natural"을 예측해야 합니다. 
- 이때 예측해야 하는 단어 "natural"을 중심단어(center word)라 하고, 예측에 사용되는 단어들을 주변 단어(context word)라고 합니다. 
- 중심 단어를 예측하기 위해 앞, 뒤로 몇 개의 단어를 볼지를 결정했다면, 그 범위를 윈도우(window)라고 합니다. 
- 만약 윈도우 크기가 1이고, 예측하고자 하는 중심 단어가 "language"라면 앞의 한 단어인 "natural"과 뒤의 한 단어인 "processing"을 참고합니다. 
- 윈도우의 크기가 m일때, 중심 단어를 예측하기 위해 참고하는 주변 단어의 개수는 2m 입니다. 
- 윈도우의 크기를 정했다면, 윈도우를 계속 움직여서 주변 단어와 중심 단어를 바꿔가며 학습을 위한 데이터 셋을 만들 수 있는데, 이 방법을 **슬라이딩 윈도우(sliding window)** 라고 합니다. 
- 윈도우의 크기가 1일때, 슬라이딩 윈도우를 처음부터 끝까지 마친다면, 다음과 같은 데이터셋을 얻을 수 있습니다.
    - 아래 데이터셋의 형식은 ((주변 단어의 셋), 중심 단어)임을 가정합니다. 
    - ((like), I), ((I, natural), like), ((like, language), natural), ((natural, processing), language), ((language), processing)
- 이렇게 선택된 데이터셋에서 단어 각각은 원-핫 인코딩되어 원-핫 벡터가 되고, 원-핫 벡터가 CBoW나 Skip-gram의 입력이 됩니다. 
- 윈도우 크기가 m이라면 2m개의 주변 단어를 이용해 1개의 중심 단어를 예측하는 과정에서 두 개의 가중치 행렬(matrix)을 학습하는 것이 목적입니다. 
- 그림에서 주황색 사각형이 첫 번째 가중치 행렬 W, 초록색 사각형이 두 번째 가중치 행렬 W' 입니다. 두 개의 가중치 행렬이 있다는 것을 인공 신경망 구조에서 보면 CBoW는 입력층, 은닉층, 출력층 이렇게 3개의 층으로만 구성된 인공신경망이라는 의미입니다. 
- Word2Vec은 은닉층이 1개이므로 딥러닝이라기보다는 얕은 신경망(Shallow Neural Network)을 학습한다고 볼 수 있습니다. 
- CBoW 신경망 구조에서 주변 단어 각각의 원-핫 벡터는 입력층에 위치하고 중심 단어의 원-핫 벡터가 위치한 곳은 출력층이라고 볼 수 있습니다. 
- CBoW에서 입력층과 출력층의 크기는 단어 집합의 크기인 V로 이미 고정되어 있습니다. 
- 하지만 은닉층의 크기는 사용자가 정의해주는 하이퍼파라미터입니다. 여기서는 은닉층의 크기는 N이라고 하겠습니다. 
- 주변 단어로 선택된 각각의 원-핫 벡터는 첫 번째 가중치 행렬과 곱해지는데, 이때 가중치 행렬의 크기는 (V x N)입니다. (V:단어집합의 크기, N:은닉층의 크기)
- 그런데 원-핫 벡터는 각 단어의 정수 인덱스 i에 해당되는 위치에만 1의 값을 가지므로, **원-핫벡터와 가중치 행렬과의 곱은 가중치 행렬의 i 위치에 있는 행을 그대로 가져오는 것과 동일** 합니다. 
- 이를 마치 테이블에서 룩업(lookup)해오는 것과 같다고 하여 **룩업 테이블(lookup table)** 이라고 합니다. 
- 룩업 테이블을 거쳐서 생긴 2m개의 주변 단어벡터들은 각각 N의 크기를 가집니다. 
- CBoW에서는 이 벡터들을 모두 합하거나, 평균을 구한 값을 최종 은닉층의 결과로 합니다. 그러면 최종 은닉층의 결과도 N차원의 벡터가 됩니다. 
- Word2Vec에서는 은닉층이나 활성화 함수나 편향(bias)을 더하는 연산을 하지 않습니다. 
- Word2Vec에서의 은닉층은 활성화 함수가 존재하지 않고, 단순히 가중치 행렬과의 곱셈만을 수행하기에 기존 신경망의 은닉층과 구분 지어 **투사층(projection layer)** 라고도 합니다. 
- 은닉층에서 생성된 N차원의 벡터는 두번째 가중치 행렬과 곱해집니다. 이 가중치 행렬의 크기는 (N x V)이므로, 곱셈의 결과로 나오는 벡터의 차원은 V입니다. 출력층은 활성화 함수로 소프트맥스 함수를 사용하므로 이 V 차원의 벡터는 활성화 함수를 거쳐 모든 차원의 총합이 1이 되는 벡터로 변경됩니다. 
- CBoW는 이 출력층의 벡터를 중심 단어의 원-핫 벡터와의 손실(loss)을 최소화하도록 학습시킵니다. 
- 이 과정에서 첫 번째 가중치 행렬 W와 두 번째 가중치 행렬 W' 가 업데이트되는데, 학습이 다 되었다면 N차원의 크기를 갖는 W의 행이나, W'의 열로부터 어떤 것을 임베딩 벡터로 사용할지를 결정하면 됩니다. 때로는 W와 W'의 평균치를 임베딩 벡터로 선택하기도 합니다.
- 참고 : [Word Embedding](https://lilianweng.github.io/lil-log/2017/10/15/learning-word-embedding.html), [What the heck is Word Embedding](https://towardsdatascience.com/what-the-heck-is-word-embedding-b30f67f01c81), [Word Embedding & GloVe](https://jonathan-hui.medium.com/nlp-word-embedding-glove-5e7f523999f6)


## 2-3. Skip-gram과 Negative Sampling
- Skip-gram은 주변 단어로 중심단어를 예측하는 것이 아니라, 중심단어로부터 주변 단어를 예측합니다. 
- 중심 단어로부터 주변 단어를 예측한다는 점, 그리고 이로 인해 중간에 은닉층에서 다수의 벡터의 덧셈과 평균을 구하는 과정이 없어졌다는 점만 제외하면 CBoW와 메커니즘 자체는 동일합니다. 
- skip-gram도 CBoW와 마찬가지로 학습 후에 가중치 행렬 W의 행렬 또는 W'의 열로부터 임베딩 벡터를 얻을 수 있습니다. 
    - 아래 데이터셋의 형식은 (중심 단어, 주변 단어)임을 가정합니다.
    - (i, like) (like, I), (like, natural), (natural, like), (natural, language), (language, natural), (language, processing), (processing, language)

- 대체적으로 Word2Vec을 사용할 때는 SGNS(Skip-Gram with Negative Sampling)을 사용합니다. 
- Word2Vec의 구조는 연산량이 지나치게 많아 실제로 사용하기 어렵기 때문입니다. 
- Skip-gram은 모델 구조는 단순해 보이지만 복잡한 과정을 거칩니다. 
- 출력층에서 소프트맥스 함수를 통과한 V 차원의 벡터와 레이블에 해당되는 V차원의 주변 단어의 원-핫 벡터와의 오차를 구하고, 역전파를 통해 모든 단어에 대한 임베딩 벡터를 조정합니다. 그 단어가 중심 단어나 주변 단어와 전혀 상관이 없는 단어라도 모두 포함하게 되어 단어장의 크기가 수십, 수백만에 달한다면 이 작업은 너무 느려지게 됩니다. 
- 그래서 네거티브 샘플링은 연산량을 줄이기 위해서 소프트맥스 함수를 사용한 V개 중 1개를 고르는 다중 클래스 분류 문제를 시그모이드 함수를 사용한 이진 분류 문제로 바꿉니다. 
- 중심 단어와 주변 단어를 입력값으로 받아 이 두 단어가 정말로 이웃 관계면 1 또는 0을 출력하는 문제로 바꾸는 것입니다. 즉, 기존의 다중 분류 문제에서 이진 분류문제로 바뀐 것입니다. 
- 예문 : Thou shalt not make a machine in the likeness of a human mind
    - 윈도우 크기가 2일 때, 위 예문으로부터 슬라이딩 윈도우를 통해 만들어지는 skip-gram 의 데이터셋에서 input word는 중심 단어, target word는 주변 단어를 의미합니다. 
    - 슬라이딩 윈도우를 통해 만들어진 정상적인 데이터셋에는 1이라는 레이블을 달아줍니다.
    - 다음으로, 랜덤으로 단어장에 있는 아무 단어나 가져와 target word로 하는 거짓 데이터셋을 만들고 0으로 레이블링을 해줍니다. 거짓(negative)데이터셋을 만들기 때문에 이 방법은 **네거티브 샘플링** 이라 불립니다. 
    - 이렇게 완성된 데이터셋으로 학습하면 Word2Vec은 더이상 다중 클래스 분류 문제가 아닌 이진 분류 문제로 간주할 수 있습니다.
    - 중심 단어와 주변 단어를 내적하고, 출력층의 시그모이드 함수를 지나게 하여 1 또는 0 레이블로부터 오차를 구해서 역전파를 수행합니다.
    - 이러한 학습 방식은 기존의 소프트맥스 함수를 사용했던 방식보다 상당량의 연산량을 줄일 수 있습니다. 이는 엄청난 연산량을 필요로 했던 Word2Vec의 학습을 가능케 했던 핵심적인 아이디어 중 하나입니다. 

## 2-4. Word2Vec 실습과 OOV 문제
- 영어 Word2Vec을 실습해봅니다. 
- Word2Vec을 별도로 구현할 필요 없이, 파이썬의 gensim 패키지를 통해 이미 구현된 Word2Vec 모델을 사용할 수 있습니다. 
- 여기서 사용할 훈련 데이터는 NLTK에서 제공하는 코퍼스이며, gensim 패키지는 토픽 모델링을 위한 NLP 패키지입니다. 
- Word2Vec은 Bag of words 학습 과정에서 언급되었던 문제인 사전에 없는 단어(Out Of Vocabulary) 문제를 그대로 가지고 있습니다. 
- 즉 사전에 없는 단어에 대해서 Word2Vec은 임베딩 벡터값을 얻을 수 없습니다. 
- 아래 실습코드에서 'overacting'은 훈련 데이터에 없는 단어(단어장에 없는 단어)이므로 에러가 발생합니다.
- 조금의 오타가 발생한 경우에도, 단어장에 없는 단어이기 때문에 error가 발생합니다.  
#### 코드 살펴보기
- Word2Vec parameter
    - vector size = 학습 후 임베딩 벡터의 차원
    - window = 컨텍스트 윈도우 크기
    - min_count = 단어 최소 빈도수 제한 (빈도가 적은 단어들은 학습하지 않음)
    - workers = 학습을 위한 프로세스 수
    - sg = 0 은 CBoW, 1은 Skip-gram

In [14]:
#- NLTK, gensim 설치
! pip install nltk
! pip install gensim

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [15]:
#- NLTK에 내장된 코퍼스 다운로드
import nltk
nltk.download('abc')
nltk.download('punkt')

[nltk_data] Downloading package abc to /aiffel/nltk_data...
[nltk_data]   Package abc is already up-to-date!
[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
#- NLTK 코퍼스를 corpus 라는 변수에 저장
from nltk.corpus import abc
corpus = abc.sents()

In [17]:
print(corpus[:3])

[['PM', 'denies', 'knowledge', 'of', 'AWB', 'kickbacks', 'The', 'Prime', 'Minister', 'has', 'denied', 'he', 'knew', 'AWB', 'was', 'paying', 'kickbacks', 'to', 'Iraq', 'despite', 'writing', 'to', 'the', 'wheat', 'exporter', 'asking', 'to', 'be', 'kept', 'fully', 'informed', 'on', 'Iraq', 'wheat', 'sales', '.'], ['Letters', 'from', 'John', 'Howard', 'and', 'Deputy', 'Prime', 'Minister', 'Mark', 'Vaile', 'to', 'AWB', 'have', 'been', 'released', 'by', 'the', 'Cole', 'inquiry', 'into', 'the', 'oil', 'for', 'food', 'program', '.'], ['In', 'one', 'of', 'the', 'letters', 'Mr', 'Howard', 'asks', 'AWB', 'managing', 'director', 'Andrew', 'Lindberg', 'to', 'remain', 'in', 'close', 'contact', 'with', 'the', 'Government', 'on', 'Iraq', 'wheat', 'sales', '.']]


In [18]:
print('코퍼스의 크기:',len(corpus))

코퍼스의 크기: 29059


In [19]:
#- Word2Vec 훈련
from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

/opt/conda/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [20]:
#- 코사인 유사도가 높은 단어 출력
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.9233418107032776), ('skull', 0.911030113697052), ('Bang', 0.905648946762085), ('asteroid', 0.9052114486694336), ('third', 0.9020071625709534), ('baby', 0.8994219303131104), ('dog', 0.898607611656189), ('bought', 0.8975202441215515), ('rally', 0.8912495374679565), ('disc', 0.8889137506484985)]


In [21]:
#- 모델 저장 및 로드
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('~/aiffel/GoingDeeper/DATA/word_embedding/w2v')
loaded_model = KeyedVectors.load_word2vec_format("~/aiffel/GoingDeeper/DATA/word_embedding/w2v")

In [22]:
#- 로드한 모델 테스트
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.9233418107032776), ('skull', 0.911030113697052), ('Bang', 0.905648946762085), ('asteroid', 0.9052114486694336), ('third', 0.9020071625709534), ('baby', 0.8994219303131104), ('dog', 0.898607611656189), ('bought', 0.8975202441215515), ('rally', 0.8912495374679565), ('disc', 0.8889137506484985)]


In [23]:
#- "Key 'overacting' not present" error
loaded_model.most_similar('overacting')

KeyError: "Key 'overacting' not present"

In [24]:
#- 오타도 용납하지 않는다.
loaded_model.most_similar('memorry')

KeyError: "Key 'memorry' not present"

# 3. 임베딩 벡터의시각화 
- 구글이 공개한 임베디 벡터의 시각화 오픈소스인 임베딩 프로젝터(embedding projector)를 사용해서 임베딩 벡터들을 시각화합니다. 
- 임베딩 프로젝터를 통해 어떤 임베딩 벡터들이 가까운 거리에 군집이 되어있고, 특정 임베딩 벡터와 유클리드 거리나 코사인 유사도가 높은지 확인할 수 있습니다. 
- 임베딩 프로젝터를 통해 임베딩 벡터를 시각화하기 위해서는 이미 저장된 모델이 필요합니다.
- 이미 저장된 모델로부터 벡터값이 저장된 파일과 메타파일을 얻어야하기 때문입니다. 
- 임베딩 프로젝터 사이트는 좌측 상단을 통해 데이터를 업로드하고, 시각화 결과를 중앙에서 볼 수 있으며, 우측에서 거리나 유사도에 대한 파라미터를 조작할 수 있는 구조입니다. 
- 우측에 Search 버튼 또는 그래프의 포인트를 클릭해 원하는 단어를 선택하고, neighbors에 몇 개까지의 이웃을 검색할지 선택합니다. distance에서 COSINE 또는 EUCLIDEAN을 통해서 거리 측정 메트릭을 코사인 유사도로 할 것인지, 유클리드 거리로 할 것인지 선택할 수 있습니다.
- 참고: [gensim scripts.word2vec2tensor](https://radimrehurek.com/gensim/models/word2vec.html), [Embedding Projector](https://projector.tensorflow.org/)

In [25]:
#- w2v_metadata.tsv와 w2v_tensor.tsv 파일 생성
! python -m gensim.scripts.word2vec2tensor --input ~/aiffel/GoingDeeper/DATA/word_embedding/w2v --output ~/aiffel/GoingDeeper/DATA/word_embedding/w2v


/opt/conda/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
2021-10-05 05:33:25,301 - word2vec2tensor - INFO - running /opt/conda/lib/python3.7/site-packages/gensim/scripts/word2vec2tensor.py --input /aiffel/aiffel/GoingDeeper/DATA/word_embedding/w2v --output /aiffel/aiffel/GoingDeeper/DATA/word_embedding/w2v
2021-10-05 05:33:25,301 - keyedvectors - INFO - loading projection weights from /aiffel/aiffel/GoingDeeper/DATA/word_embedding/w2v
2021-10-05 05:33:26,292 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (10363, 100) matrix of type float32 from /aiffel/aiffel/GoingDeeper/DATA/word_embedding/w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-10-05T05:33:26.286971', '

# 4. FastText
- 페이스북에서 개발한 FastText는 Word2Vec이후에 등장한 워드 임베딩 방법입니다. 
- 메커니즘 자체는 Word2Vec을 그대로 따르지만, 문자 단위 n-gram(chacracter-level n-gram)표현을 학습한다는 점에서 다릅니다. 
- Word2Vec은 단어를 더 이상 깨질 수 없는 단위로 구분하는 반면, FastText는 단어 내부의 내부단어(subwords)들을 학습한다는 아이디어를 가지고 있습니다. 
- FastText의 n-gram에서 n은 단어들이 얼마나 분리되는지 결정하는 하이퍼파라미터입니다.
- n을 3으로 잡은 트라이그램(tri-gram)의 경우, 단어 "partial"은 'par','art','rti','tia','ial'로 분리하고 이들을 벡터로 만듭니다.
- 더 정확히는 시작과 끝을 의미하는 \<,\>를 도입하여 \<pa, are, rti, tia, ial, al\> 라는 6개의 내부 단어(subword) 토큰을 벡터로 만듭니다. 여기에 추가로 기존 단어에 \<,\> 를 붙인 토큰 \<partial\>를 벡터화합니다.
- 즉 n=3인 경우, FastText는 단어 partial 에 대해 임베딩되는 n-gram 토큰은 다음과 같습니다. 
    - \<pa, art, rti, tia, ial, al\>, \<partial\>
- 실제 사용시에는 n의 최솟값, 최댓값으로 범위를 설정할 수 있는데, gensim 패키지에서는 기본값으로 각각 3과 6으로 설정되어 있습니다. 
    - \<pa, art, rti, ita, ial, al\>, \<par, arti, rtia, tial, ial\>, \<part, ...중략... , \<partial\>

- 내부단어들을 벡터화한다는 의미는 위 단어들 각각에 대해서 Word2Vec을 수행한다는 의미입니다. 최종적으로 벡터화된 n-gram 벡터들의 총합을 해당 단어의 벡터로 취합니다.
    - partial = \<pa + art + rti + ita + ial + a\l> + \<par + arti + rtia + tial + ia\l> + \<part + ...중략...  + \<partial\>

### FastText의 학습방법
- FastText도 Word2Vec과 마찬가지로 네거티브 샘플링을 사용하여 학습합니다. 
- "(중심 단어, 주변 단어)"의 쌍을 가지고 이 쌍이 포지티브인지 네거티브인지 예측을 진행하는 것입니다. 
- 다만, Word2Vec와 다른 점은 학습 과정에서 중심 단어에 속한 문자 단위 n-gram 단어 벡터들을 모두 업데이트한다는 점입니다.

### OOV 와 오타에 대한 대응
- FastText는 Word2Vec과 달리 OOV와 오타에 robust(강건한)한 특징이 있습니다.
- 이는 단어장에 없는 단어라도, 해당 단어의 n-gram이 다른 단어에 존재하면 이로부터 벡터값을 얻는다는 원리에 기인합니다. 

### 한국어에서의 FastText
- 영어의 경우 문자, 즉 알파벳 단위가 n-gram이었다면 한국어의 경우는 음절 단위라고 볼 수 있습니다. 
- 한국어 어휘를 자모 수준으로 분리하여 학습하는 것은 일부 어휘의 특성을 학습하는 데 큰 도움이 되지 않거나 가끔은 성능의 저하를 일으키기 때문에, 아보카도와 같은 고유 명사는 분해해도 이득을 볼 수 없습니다.

1. 음절 단위 FastText
- n=3일때 단어 '아보카도씨앗'의 트라이그램 벡터
    - \<아보, 아보카, 보카도, 카도씨, 도씨앗, 씨앗\>
- 한국어에서 FastText가 빛을 발하는 것은 음절 단위보다는 자소 단위입니다. 
- 한국어의 자소를 각각의 문자로 간주한 경우, FastText는 꽤 잘 동작한다고 알려져있습니다.

2. 자소 단위 FastText
- 단어에 대해서 초성, 중성, 종성을 분리한다고 하고, 종성이 존재하지 않는 경우에 _ 라는 토큰을 사용
    - \<ㅇㅏ,ㅇㅏ_,ㅏ_ㅂ,중략..\>

- 참고 : [한국어 어휘임베딩](https://brunch.co.kr/@learning/8)

In [26]:
from gensim.models import FastText
fasttext_model = FastText(corpus, window=5, min_count=5, workers=4, sg=1)

In [27]:
fasttext_model.wv.most_similar('overacting')

[('fluctuating', 0.9379985332489014),
 ('resolving', 0.9358386397361755),
 ('emptying', 0.9348052740097046),
 ('mounting', 0.9336827397346497),
 ('malting', 0.9332555532455444),
 ('extracting', 0.9324961304664612),
 ('shooting', 0.9301201105117798),
 ('debilitating', 0.9300547242164612),
 ('overwhelming', 0.9289087057113647),
 ('attracting', 0.9270631670951843)]

In [28]:
fasttext_model.wv.most_similar('memorry')

[('memory', 0.9389050006866455),
 ('xenotourism', 0.9263287782669067),
 ('consciousness', 0.9246035218238831),
 ('tertiary', 0.909155011177063),
 ('counterintuitive', 0.9087101221084595),
 ('tourism', 0.9081308245658875),
 ('terrorism', 0.9055162072181702),
 ('unrealistic', 0.9029677510261536),
 ('jealousy', 0.9014033079147339),
 ('memorandum', 0.9008081555366516)]

# 5. GloVe
- 글로브(Gloval Vectors for Word Representation, GloVe)는 2014년에 미국 스탠포드 대학에서 개발한 워드 임베딩 방법론입니다. 

- 워드 임베딩의 두 가지 접근 방법인 카운트 기반과 예측 기반 두 가지 방법을 모두 사용했다는 것이 특징입니다.
-  행렬 A에 대해서 특이값 분해의 변형인 thin SVD, compact SVD, Truncated SVD를 각각 사용하였을 때 thin SVD, compact SVD 두 가지는 행렬 A를 복원할 수 있습니다.
- 잠재 의미 분석(LSA, Latent Semantic Analysis)은 DTM에 특잇값 분해를 사용하여 잠재된 의미를 이끌어내는 방법론입니다. 
- LSA는 단어를 카운트해서 만든 DTM을 입력으로 하므로 카운트 기반의 임베딩 방법이라고 볼 수 있는데, 이 방법은 몇 가지 한계가 있었습니다.
    - 차원 축소의 특성으로 인해 새로운 단어가 추가되면 다시 DTM을 만들어 새로 차원축소를 해야 합니다.
    - 단어 벡터간 유사도를 계산하는 측면에서 Word2Vec보다 성능이 떨어집니다. 
- 반면 LSA와 대조되는 방법으로 예측 기반의 방법은 Word2Vec과 같은 방법입니다. 
- Word2Vec은 인공신경망이 에측한 값으로부터 실제 레이블과의 오차를 구하고, 손실함수를 통해서 인공 신경망을 학습하는 방법입니다. 
- GloVe 연구진은 Word2Vec의 경우에는 LSA보다 단어 벡터 간 유사도를 구하는 능력은 뛰어나지만, LSA처럼 코퍼스의 전체적인 통계 정보를 활용하지 못한다는 점을 한계로 지적했습니다. 
- 그리고 카운트 기반과 예측 기반을 모두 사용하여 Word2Vec보다 더 나은 임베딩 방법을 GloVe를 제안합니다. 

### 윈도우 기반 동시 등장 행렬(Window based Co-occurrence Matrix)
- 윈도우 기반 동시 등장 행렬은 행과 열을 전체 단어장(vocabulary)의 단어들로 구성하고, 어떤 i 단어의 윈도우 크기(window Size)내에서 k 단어가 등장한 횟수를 i행 k열에 기재한 행렬입니다. 
- 동시 등장 행렬은 전치(transpose)해도 동일한 행렬이 된다는 특징을 가지고 있습니다.

### 동시 등장 확률(Co-occurrence Probability)
- 동시 등장 확률 P(k∣i)는 동시 등장 행렬로부터 특정 단어 i의 전체 등장 횟수를 카운트하고, 특정 단어 i가 등장했을 때 어떤 단어 k가 등장한 횟수를 카운트하여 계산한 조건부 확률입니다.
- 이때 i를 중심 단어(center word), k를 주변 단어(context word)입니다.

### GloVe의 손실 함수 설계하기
- GloVe는 동시 등장 행렬로부터 계산된 동시 등장 확률을 이용해 손실 함수를 설계합니다. 
- 동시 등장 행렬을 사용하고 있으니 코퍼스의 전체적인 통계 정보를 활용하는 '카운트 기반'의 방법론이면서, 손실 함수를 통해 모델을 학습시키므로 '예측 기반'의 방법론이라고 할 수 있는 것입니다.
- GloVe는 중심 단어 벡터와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 빈도의 로그값이 되도록 만든다는 아이디어입니다. 
- 다시말하면, 전체 코퍼스에서의 동시 등장 빈도의 로그값과 중심 단어 벡터와 주변 단어 벡터의 내적값의 차이가 최소화되도록 두 벡터의 값을 학습하는 것입니다. 


- 참고 : [SVD, PCA, LSA](https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/04/06/pcasvdlsa/), [LSA 설명영상](https://youtu.be/GVPTGq53H5I), [동시등장행렬](http://web.stanford.edu/class/cs224n/slides/cs224n-2019-lecture02-wordvecs2.pdf), [글로브](https://wikidocs.net/22885)

In [34]:
# ! conda create -n glove python=3.6
# ! pip install glove_python_binary
# ! pip install nltk

In [30]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to /aiffel/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
from nltk.corpus import movie_reviews
corpus=movie_reviews.sents()

In [32]:
from glove import Corpus, Glove

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
emb = Corpus() 
emb.fit(corpus, window=5)

# 벡터의 차원은 100, 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(emb.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(emb.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [33]:
model_result1 = glove.most_similar("man")
model_result2 = glove.most_similar("fiction")

print("model_result1", model_result1)
print("model_result2", model_result2)

model_result1 [('woman', 0.9461619944822177), ('young', 0.8942342905325987), ('boy', 0.8773305685655169), ('girl', 0.8710872211056477)]
model_result2 [('science', 0.9835459895915012), ('pulp', 0.9704991745128316), ('kong', 0.7003685929466086), ('rocky', 0.6954276239990997)]
